<a href="https://colab.research.google.com/github/douglasmmachado/MedicineConsumption/blob/main/notebooks/unified_approach/5_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5 - Forecasting and prediction



---



---



In [148]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import math as m

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

from sklearn.tree import export_graphviz
from subprocess import call
from IPython.display import Image

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error,  mean_absolute_percentage_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV



df_3_clusters_url ="https://raw.githubusercontent.com/douglasmmachado/MedicineConsumption/main/datasets/unified_approach/clustered/df_clustered_3.csv"
df_4_clusters_url ="https://raw.githubusercontent.com/douglasmmachado/MedicineConsumption/main/datasets/unified_approach/clustered/df_clustered_4.csv"
df_5_clusters_url ="https://raw.githubusercontent.com/douglasmmachado/MedicineConsumption/main/datasets/unified_approach/clustered/df_clustered_5.csv"
df_url = "https://raw.githubusercontent.com/douglasmmachado/ExploratoryDataAnalysis/main/datasets/unified_approach/df_ma.csv"

df = pd.read_csv(df_url)
df_3_clusters = pd.read_csv(df_3_clusters_url)
df_4_clusters = pd.read_csv(df_4_clusters_url)
df_5_clusters = pd.read_csv(df_5_clusters_url)

# Convert numerical columns to string
for i, curr_df in enumerate([df, df_3_clusters, df_4_clusters, df_5_clusters]):
    numerical_columns = ['HOSPI_CODE_UCD']
    curr_df[numerical_columns] = curr_df[numerical_columns].astype('string')


medicines = ['3400892088310','3400892075761','3400892203645',
             '3400892065366','3400892052120','3400891996128',
             '3400893826706','3400893736135','3400893875490',
             '3400890837149','3400891235203','3400891225037',
             '3400891191226','3400892729589','3400892745848',
             '3400892697789','3400892761527','3400893022634',
             '3400892761695','3400892669236','3400892508566']

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4074 entries, 0 to 4073
Data columns (total 48 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   HOSPI_CODE_UCD     4074 non-null   string 
 1   LIT_HC             4074 non-null   float64
 2   LIT_HP             4074 non-null   float64
 3   N_ETB              4074 non-null   float64
 4   N_UFS              4074 non-null   float64
 5   PN_MEDICAL         4074 non-null   float64
 6   POPULATION         4074 non-null   float64
 7   P_MEDICAL          4074 non-null   float64
 8   QUANTITY           4074 non-null   float64
 9   QUANTITY_MA        4074 non-null   float64
 10  SEJ_HAD            4074 non-null   float64
 11  SEJ_MCO            4074 non-null   float64
 12  SEJ_PSY            4074 non-null   float64
 13  SEJ_SLD            4074 non-null   float64
 14  SEJ_SSR            4074 non-null   float64
 15  MONTH_1.0          4074 non-null   int64  
 16  MONTH_2.0          4074 

## 5.1 - New database composition based on clusters

## 5.2 - Building forecasting models based on clusters

In [168]:
import plotly.graph_objects as go

def plot_pred(y_pred, y_test, medicine):
    # Create a scatter plot for y_pred
    fig = go.Figure(data=go.Scatter(x=np.arange(len(y_pred)), y=y_pred, mode='markers', name='y_pred', marker=dict(symbol='circle')))

    # Add scatter plot for y_test
    fig.add_trace(go.Scatter(x=np.arange(len(y_test)), y=y_test, mode='markers', name='y_test', marker=dict(symbol='x')))

    # Set axes labels and title
    fig.update_layout(xaxis_title='Test samples', yaxis_title='Quantity',
                      title=f'y_pred and y_test for medicine: {medicine}')

    # Show the plot
    fig.show()


In [166]:
import plotly.graph_objects as go

def plot_mape(y_pred, y_test, medicine, epsilon=0.001):
    mape_array = np.abs(y_test - y_pred) / np.maximum(epsilon, np.abs(y_test))

    # Create a scatter plot for MAPE
    fig = go.Figure(data=go.Scatter(x=np.arange(len(y_pred)), y=mape_array, mode='markers', name='MAPE'))

    # Set axes labels and title
    fig.update_layout(xaxis_title='Test samples', yaxis_title='MAPE',
                      title=f'MAPE for medicine: {medicine}')

    # Show the plot
    fig.show()


In [185]:
def test_1_baseline(df, medicine, df_scores):
  print()
  print(100*'-')
  print('Medicine:' + str(medicine))

  X = df[df['HOSPI_CODE_UCD'] == medicine].drop(['QUANTITY', 'HOSPI_CODE_UCD'], axis=1).values

  scaler = StandardScaler()
  X_scaled = scaler.fit_transform(X)

  y = df[df['HOSPI_CODE_UCD'] == medicine]['QUANTITY'].values

  X_scaled, y = shuffle(X_scaled, y, random_state=42)

  if m.ceil(len(X_scaled) * 0.1) == 1:
    print('Dataset too small')
    test_size = 2
  else:
    test_size = 0.1

  # Split the data into training and testing sets
  X_train, X_test, y_train, y_test = train_test_split(X_scaled, y,
                                                      test_size = test_size,
                                                      random_state = 42)
  print(f'Size of data set: {len(X)}')
  print(f'Size of training set: {len(X_train)}')
  print(f'Size of test set: {len(X_test)}')

  # Define the parameter distributions for RandomizedSearchCV
  param_grid = {
      'max_depth': np.arange(2, 8, 1),
      'n_estimators': np.arange(2, max(int(m.ceil(len(X_train)*0.1)),3), 1),
      'max_features': ['sqrt']
  }
  depth_len = param_grid['max_depth'].size
  estimators_len = param_grid['n_estimators'].size

  print(f'Size of grid search: {depth_len * estimators_len}')

  # Create the RandomizedSearchCV object
  grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=42),
                             param_grid=param_grid,
                             scoring = 'neg_mean_absolute_percentage_error',
                             cv = 5,
                             n_jobs = -1)

  ''' >3 : the fold and candidate parameter indexes
      are also displayed together with the starting time of the computation.
  '''
  # Fit the RandomizedSearchCV object to the data
  grid_search.fit(X_train, y_train)

  # Get the best estimator
  best_estimator = grid_search.best_estimator_

  # Make predictions using the best estimator
  y_pred = best_estimator.predict(X_test)

  # Calculate R^2 score
  r2 = r2_score(y_test, y_pred)

  # Calculate MAE
  mae = mean_absolute_error(y_test, y_pred)

  # Calculate MAPE
  mape = mean_absolute_percentage_error(y_test, y_pred)

  # Calculate RMSE
  rmse = np.sqrt(mean_squared_error(y_test, y_pred))

  # Print the best parameters, best score, and evaluation metrics
  print('Best Parameters: ', grid_search.best_params_)
  print('Training Score (MAPE): ', round(grid_search.best_score_, 3))
  print(10*'-' + 'Test scores' + 10*'-')
  print('R^2 Score:', round(r2, 3))
  print('MAE:', round(mae, 3))
  print('MAPE:', round(mape, 3))
  print('RMSE:', round(rmse, 3))
  print()


  # Create the new row as a DataFrame
  new_row = pd.DataFrame({'HOSPI_CODE_UCD': ['CODE_UCD_'+str(medicine)],
                          'R2': [r2],
                          'RMSE': [rmse],
                          'MAE': [mae],
                          'MAPE': [mape]})

  # Append the new row to the DataFrame
  df_scores = pd.concat([df_scores, new_row], ignore_index=True)

  # plot pred x test
  plot_pred(y_pred, y_test, medicine)
  print()


  plot_mape(y_pred, y_test, medicine)
  print()

  # Return the updated DataFrame
  return df_scores


In [188]:
features = ['HOSPI_CODE_UCD', 'LIT_HC', 'LIT_HP', 'N_ETB', 'N_UFS', 'PN_MEDICAL',
       'POPULATION', 'P_MEDICAL', 'QUANTITY', 'SEJ_HAD',
       'SEJ_MCO', 'SEJ_PSY', 'SEJ_SLD', 'SEJ_SSR', 'MONTH_1.0', 'MONTH_2.0',
       'MONTH_3.0', 'MONTH_4.0', 'MONTH_5.0', 'MONTH_6.0', 'MONTH_7.0',
       'MONTH_8.0', 'MONTH_9.0', 'MONTH_10.0', 'MONTH_11.0', 'MONTH_12.0', 'CLUSTER']

features_2 = ['HOSPI_CODE_UCD', 'LIT_HC', 'LIT_HP', 'N_ETB', 'N_UFS', 'PN_MEDICAL',
       'POPULATION', 'P_MEDICAL', 'QUANTITY', 'SEJ_HAD',
       'SEJ_MCO', 'SEJ_PSY', 'SEJ_SLD', 'SEJ_SSR', 'MONTH_1.0', 'MONTH_2.0',
       'MONTH_3.0', 'MONTH_4.0', 'MONTH_5.0', 'MONTH_6.0', 'MONTH_7.0',
       'MONTH_8.0', 'MONTH_9.0', 'MONTH_10.0', 'MONTH_11.0', 'MONTH_12.0']

In [189]:
df_prediction_scores = pd.DataFrame(columns=['HOSPI_CODE_UCD', 'R2', 'RMSE', 'MAE', 'MAPE'])

for medicine in medicines:
  df_prediction_scores = test_1_baseline(df[features_2], medicine, df_prediction_scores)

df_prediction_scores


----------------------------------------------------------------------------------------------------
Medicine:3400892088310
Size of data set: 171
Size of training set: 153
Size of test set: 18
Size of grid search: 84
Best Parameters:  {'max_depth': 6, 'max_features': 'sqrt', 'n_estimators': 2}
Training Score (MAPE):  -3.172
----------Test scores----------
R^2 Score: 0.432
MAE: 865.071
MAPE: 0.275
RMSE: 1449.716





----------------------------------------------------------------------------------------------------
Medicine:3400892075761
Size of data set: 204
Size of training set: 183
Size of test set: 21
Size of grid search: 102
Best Parameters:  {'max_depth': 7, 'max_features': 'sqrt', 'n_estimators': 5}
Training Score (MAPE):  -0.236
----------Test scores----------
R^2 Score: 0.287
MAE: 480.302
MAPE: 0.19
RMSE: 701.564





----------------------------------------------------------------------------------------------------
Medicine:3400892203645
Size of data set: 200
Size of training set: 180
Size of test set: 20
Size of grid search: 96
Best Parameters:  {'max_depth': 7, 'max_features': 'sqrt', 'n_estimators': 6}
Training Score (MAPE):  -0.735
----------Test scores----------
R^2 Score: 0.923
MAE: 751.914
MAPE: 1.165
RMSE: 1096.752





----------------------------------------------------------------------------------------------------
Medicine:3400892065366
Size of data set: 201
Size of training set: 180
Size of test set: 21
Size of grid search: 96
Best Parameters:  {'max_depth': 7, 'max_features': 'sqrt', 'n_estimators': 3}
Training Score (MAPE):  -1.162
----------Test scores----------
R^2 Score: 0.619
MAE: 794.494
MAPE: 0.841
RMSE: 1390.346





----------------------------------------------------------------------------------------------------
Medicine:3400892052120
Size of data set: 195
Size of training set: 175
Size of test set: 20
Size of grid search: 96
Best Parameters:  {'max_depth': 6, 'max_features': 'sqrt', 'n_estimators': 6}
Training Score (MAPE):  -1.013
----------Test scores----------
R^2 Score: 0.821
MAE: 284.008
MAPE: 4.292
RMSE: 442.054



...



----------------------------------------------------------------------------------------------------
Medicine:3400893022634
Size of data set: 157
Size of training set: 141
Size of test set: 16
Size of grid search: 78
Best Parameters:  {'max_depth': 6, 'max_features': 'sqrt', 'n_estimators': 2}
Training Score (MAPE):  -14.272
----------Test scores----------
R^2 Score: 0.374
MAE: 1109.167
MAPE: 13.178
RMSE: 1432.724





----------------------------------------------------------------------------------------------------
Medicine:3400892761695
Size of data set: 201
Size of training set: 180
Size of test set: 21
Size of grid search: 96
Best Parameters:  {'max_depth': 7, 'max_features': 'sqrt', 'n_estimators': 4}
Training Score (MAPE):  -2.525
----------Test scores----------
R^2 Score: 0.977
MAE: 1221.255
MAPE: 2.535
RMSE: 1624.762





----------------------------------------------------------------------------------------------------
Medicine:3400892669236
Size of data set: 173
Size of training set: 155
Size of test set: 18
Size of grid search: 84
Best Parameters:  {'max_depth': 5, 'max_features': 'sqrt', 'n_estimators': 6}
Training Score (MAPE):  -0.095
----------Test scores----------
R^2 Score: -0.641
MAE: 851.322
MAPE: 0.087
RMSE: 1373.509





----------------------------------------------------------------------------------------------------
Medicine:3400892508566
Size of data set: 191
Size of training set: 171
Size of test set: 20
Size of grid search: 96
Best Parameters:  {'max_depth': 3, 'max_features': 'sqrt', 'n_estimators': 2}
Training Score (MAPE):  -1.248
----------Test scores----------
R^2 Score: 0.739
MAE: 869.766
MAPE: 0.454
RMSE: 1134.489



,HOSPI_CODE_UCD,R2,RMSE,MAE,MAPE
0,CODE_UCD_3400892088310,0.431571,1449.715750,865.071363,0.275251
1,CODE_UCD_3400892075761,0.287241,701.563925,480.301518,0.190485
2,CODE_UCD_3400892203645,0.923381,1096.751736,751.914462,1.164690
3,CODE_UCD_3400892065366,0.619334,1390.345977,794.494012,0.841362
4,CODE_UCD_3400892052120,0.820951,442.053982,284.007655,4.291859
5,CODE_UCD_3400891996128,0.866556,12190.333569,8865.497828,34.926425
6,CODE_UCD_3400893826706,0.798565,1246.543840,950.756882,0.517109
7,CODE_UCD_3400893736135,0.853041,891.155733,623.069031,0.122767
8,CODE_UCD_3400893875490,0.732961,2489.624688,1549.652675,0.624151
9,CODE_UCD_3400890837149,0.607444,4246.565875,1956.425962,24.676388


In [190]:
def train_test_split_modified(df, random_state = 42, test_size = 0.1, Shuffle = False):
  df_iter = df.copy()

  np.random.seed(random_state)

  X_train = []
  y_train = []

  X_test = []
  y_test = []

  columns = df.columns

  df_train = pd.DataFrame(columns=columns)
  df_test = pd.DataFrame(columns=columns)
  i = 0
  for medicine in df_iter.HOSPI_CODE_UCD.unique():
    df_temp = df_iter[df_iter['HOSPI_CODE_UCD'] == medicine]
    i += len(df_temp)
    n_samples = len(df_temp)
    test_samples = max(m.ceil(n_samples * test_size), 2)

    for _ in range(test_samples):
      random_index = np.random.choice(df_temp.index)
      random_row = df_temp.loc[random_index].to_frame().T

      # Concatenate the new row to the original DataFrame
      df_test = pd.concat([df_test, random_row], ignore_index=False)

      # Remove the selected index from df_temp
      df_temp = df_temp.drop(random_index)

    df_iter = df_iter.drop(df_iter[df_iter['HOSPI_CODE_UCD'] == medicine].index)

  # Get the indices of rows to exclude from df
  exclude_indices = df_test.index

  # Create df_train by dropping the rows specified by the exclude_indices from df
  df_train = df.drop(exclude_indices)

  if shuffle:
    X_train, y_train = shuffle(df_train.drop(['QUANTITY', 'QUANTITY_MA', 'CLUSTER'], axis=1), df_train.QUANTITY.values, random_state = random_state)
    X_test, y_test = shuffle(df_test.drop(['QUANTITY', 'QUANTITY_MA', 'CLUSTER'], axis=1).values, df_test.QUANTITY.values, random_state = random_state)

  return X_train, X_test, y_train, y_test

In [191]:
def test_2_clustering(df, df_scores, medicines):
  for cluster in df.CLUSTER.unique():
    print()
    print(100*'-')
    print(f'Cluster: {cluster}')

    # Perform the train-test split with shuffled samples
    X_train, X_test, y_train, y_test = train_test_split_modified(df[df['CLUSTER'] == cluster])
    print(f'Size of data set: {len(X)}')
    print(f'Size of training set: {len(X_train)}')
    print(f'Size of test set: {len(X_test)}')

    df_test = pd.DataFrame(X_test, columns = df.drop(['QUANTITY', 'CLUSTER'], axis=1).copy().columns)
    df_test['QUANTITY'] = y_test

    # Define the parameter distributions for RandomizedSearchCV
    param_grid = {
        'max_depth': np.arange(2, 8, 1),
        'n_estimators': np.arange(2, max(int(m.ceil(len(X_train)*0.1)),3), 1),
        'max_features': ['sqrt']
    }

    depth_len = param_grid['max_depth'].size
    estimators_len = param_grid['n_estimators'].size

    print(f'Size of grid search: {depth_len * estimators_len}')

    # Create the RandomizedSearchCV object
    grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=42),
                              param_grid=param_grid,
                              scoring = 'neg_mean_absolute_percentage_error',
                              cv = 5,
                              n_jobs = -1)

    ''' >3 : the fold and candidate parameter indexes
        are also displayed together with the starting time of the computation.
    '''

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    # Fit the RandomizedSearchCV object to the data
    grid_search.fit(X_train_scaled, y_train)
    print("Finished training")

    # Get the best estimator
    best_estimator = grid_search.best_estimator_

    for medicine in df_test.HOSPI_CODE_UCD.unique():
      print()
      print(100*'-')
      print('Medicine:' + str(medicine))

      X_test_medicine = df_test[df_test['HOSPI_CODE_UCD'] == medicine].drop(['QUANTITY'], axis=1).copy().values

      scaler = StandardScaler()
      X_test_scaled = scaler.fit_transform(X_test_medicine)

      y_test_medicine = df_test[df_test['HOSPI_CODE_UCD'] == medicine]['QUANTITY'].copy().values

      # Make predictions using the best estimator
      y_pred = best_estimator.predict(X_test_scaled)

      # Calculate R^2 score
      r2 = r2_score(y_test_medicine, y_pred)

      # Calculate MAE
      mae = mean_absolute_error(y_test_medicine, y_pred)

      # Calculate MAPE
      mape = mean_absolute_percentage_error(y_test_medicine, y_pred)

      # Calculate RMSE
      rmse = np.sqrt(mean_squared_error(y_test_medicine, y_pred))

      # Print the best parameters, best score, and evaluation metrics

      # Print the best parameters, best score, and evaluation metrics
      print('Best Parameters: ', grid_search.best_params_)
      print('Training Score (MAPE): ', round(grid_search.best_score_, 3))
      print(10*'-' + 'Test scores' + 10*'-')
      print('R^2 Score:', round(r2, 3))
      print('MAE:', round(mae, 3))
      print('MAPE:', round(mape, 3))
      print('RMSE:', round(rmse, 3))
      print()


      # Create the new row as a DataFrame
      new_row = pd.DataFrame({'CLUSTER': [cluster],
                              'HOSPI_CODE_UCD': ['CODE_UCD_'+str(int(medicine))],
                              'R2': [r2],
                              'RMSE': [rmse],
                              'MAE': [mae],
                              'MAPE': [mape]})

      # Append the new row to the DataFrame
      df_scores = pd.concat([df_scores, new_row], ignore_index=True)

      plot_pred(y_pred, y_test_medicine, medicine)
      print()

      plot_mape(y_pred, y_test_medicine, medicine)
      print()

  # Return the updated DataFrame
  return df_scores

In [192]:

df_prediction_scores_3_clusters = pd.DataFrame(columns=[ 'HOSPI_CODE_UCD', 'CLUSTER', 'R2', 'RMSE', 'MAE', 'MAPE'])

df_prediction_scores_3_clusters = test_2_clustering(df_3_clusters[features], df_prediction_scores_3_clusters, medicines)

df_prediction_scores_3_clusters


----------------------------------------------------------------------------------------------------
Cluster: 0


KeyError: ignored

In [ ]:
df_prediction_scores_4_clusters = pd.DataFrame(columns=[ 'HOSPI_CODE_UCD', 'CLUSTER', 'R2', 'RMSE', 'MAE', 'MAPE'])

df_prediction_scores_4_clusters = test_2_clustering(df_4_clusters[features], df_prediction_scores_4_clusters, medicines)

df_prediction_scores_4_clusters

In [ ]:
df_prediction_scores_5_clusters = pd.DataFrame(columns=[ 'HOSPI_CODE_UCD', 'CLUSTER', 'R2', 'RMSE', 'MAE', 'MAPE'])

df_prediction_scores_5_clusters = test_2_clustering(df_5_clusters[features], df_prediction_scores_5_clusters, medicines)

df_prediction_scores_5_clusters